In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
import datetime
import yfinance as yf
from itertools import product

### Big moves Monday
Buy on Monday given a few conditions and exit on Friday
1. Cond No 1: Friday Close - Monday Close  >= 0.25 *  Relative Range MA
2. IBS must be lower than 0.3
3. The dau must be a Monday; 1 day before has to a Friday: 4 day after is Friday.

- Relative Range = (HIgh - Low)/ Close
- IBS = (Close - Low)/(High - Low)

In [5]:
class Financial_data():
    def __init__(self, ticker, start, end):
        self.ticker = ticker
        self.start = start
        self.end = end
        
        self.df= yf.download(self.ticker, self.start, self.end)
        self.df= pd.DataFrame(self.df)
        self.df['r'] = np.log(self.df['Adj Close']/self.df['Adj Close'].shift(1))
        self.df.dropna(inplace=True)
    

In [31]:
class BMM(Financial_data):
    def BMM_backtest(self, avg_days=None):
        if avg_days is None:
            avg_days = 14
        self.avg_days = avg_days
        
        self.indicators()
        self.signal()
        
        self.df['s'] = self.df['position'].shift(1)* self.df['r']
        perf = np.exp(self.df[['r','s']].iloc[self.avg_days:].sum())
        return perf
    
    def indicators(self):
        # Create Relative Range MA
        self.df['RR'] = (self.df['High'] - self.df['Low'])/self.df['Close']
        self.df['RR_ma'] = self.df['RR'].rolling(window= self.avg_days).mean()
        
        # Create IBS
        self.df['IBS'] = (self.df['Adj Close']- self.df['Low'])/(self.df['High']- self.df['Low'])
        
        # Create indicators for days
        self.df['day'] = self.df.index.day_name()
        self.df['prev_day'] = self.df['day'].shift(1)
        self.df['4_days_later'] = self.df['day'].shift(-4)
        
    def signal(self):
        #Signal 1 : RR_MA
        self.df['cond_1'] = np.where((self.df['Adj Close'].shift(1)- self.df['Adj Close'])>= 0.25*self.df['RR_ma'], 1,0)
        
        #signal 2: IBS
        self.df['cond_2'] = np.where(self.df['IBS']< 0.3, 1,0)
        
        # signal 3: Day
        self.df['cond_3'] = np.where((self.df['day']== 'Monday')
                                     & (self.df['prev_day']== 'Friday')
                                     & (self.df['4_days_later'] == 'Friday'),
                                     1,0)
        
        self.df['signal'] = np.where((self.df['cond_1']==1)
                                    & (self.df['cond_2']==1)
                                    & (self.df['cond_3']==1), 1, 0)
        self.df['position'] = self.df['signal'].replace(to_replace=0, method='ffill', limit=3)
        
        

In [32]:
end = datetime.date.today()
start= end- pd.Timedelta(days=3*365)

In [33]:
bmm = BMM("^NSEI", start, end)

[*********************100%***********************]  1 of 1 completed


In [34]:
bmm.BMM_backtest(20)

r    1.578846
s    1.165111
dtype: float64

In [35]:
# bmm.df.to_csv('BMM.csv')

In [37]:
days=[10,14,21,30]

In [45]:
res = pd.DataFrame()
for i in days:
    
    perf = bmm.BMM_backtest(i)
    res = res.append(pd.DataFrame({'No of Days':i, 'Benchmark Ret': perf['r'], 'Strat_ret': perf['s']}, index=[0]), ignore_index=True) 
    #print(perf)
print(res)

   No of Days  Benchmark Ret  Strat_ret
0          10       1.518627   1.165111
1          14       1.518809   1.165111
2          21       1.597357   1.165111
3          30       1.503504   1.138662


In [46]:
res

,No of Days,Benchmark Ret,Strat_ret
0,10,1.518627,1.165111
1,14,1.518809,1.165111
2,21,1.597357,1.165111
3,30,1.503504,1.138662
